In [1]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

In [2]:
import pymc3 as pm

In [3]:
np.random.seed(23) # from random.org

# 3.1. Individual GLAM estimation and out of sample prediction

## Load data

In [4]:
# Load data
sufix = '_individual_More_NoBin_Gamma-11_NUTS_33'
data = pd.read_csv('data/PF2019_data/GlamDataPF2019_More_NoBin_33.csv')

# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]
data.head()

subject  trial  choice        rt  item_value_0  item_value_1    gaze_0  \
0        1      0       1  1734.284           110           131  0.669090   
1        1      1       0  6555.370            47            50  0.759630   
2        1      2       0  3174.566            50            44  0.549371   
3        1      3       1  2877.579            57            50  0.608409   
4        1      4       1  1806.310            42            50  0.522849   

     gaze_1  
0  0.330910  
1  0.240370  
2  0.450629  
3  0.391591  
4  0.477151

## Split data in training and test sets

In [5]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for subject in data.subject.unique():
    subject_data = data[data['subject'] == subject].copy().reset_index(drop=True)
    n_trials = len(subject_data)
    
    subject_train = subject_data.iloc[np.arange(0, n_trials, 2)].copy()
    subject_test = subject_data.iloc[np.arange(1, n_trials, 2)].copy()

    test_data = pd.concat([test_data, subject_test])
    train_data = pd.concat([train_data, subject_train])

test_data.to_csv(str('data/PF2019_data/GlamDataPF2019_preprocessed_test'+sufix+'.csv'))
train_data.to_csv(str('data/PF2019_data/GlamDataPF2019_preprocessed_train'+sufix+'.csv'))

print('Split data into training ({} trials) and test ({} trials) sets...'.format(len(train_data), len(test_data)))

Split data into training (1920 trials) and test (1920 trials) sets...


In [6]:
train_data

subject  trial  choice         rt  item_value_0  item_value_1    gaze_0  \
0          1      0       1   1734.284           110           131  0.669090   
2          1      2       0   3174.566            50            44  0.549371   
4          1      4       1   1806.310            42            50  0.522849   
6          1      6       1   3650.266            78            80  0.682034   
8          1      8       1   1259.268            50            48  0.508019   
10         1     10       1   5698.027            46            50  0.582089   
12         1     12       1   2626.168            50            56  0.666353   
14         1     14       1   1671.149           110           124  0.508035   
16         1     16       1   1527.826            64            80  0.473512   
18         1     18       1   2267.665           101           110  0.553242   
20         1     20       0   5571.061            50            42  0.575306   
22         1     22       1   1402.112            77            80  0.435523   
24         1     24       1   2065.604            44            50  0.528335   
26         1     26       1   1613.454            50            46  0.485035   
28         1     28       0   4895.517           133           110  0.698379   
30         1     30       1   2000.688           126           110  0.511292   
32         1     32       1   2050.497           110           126  0.522393   
34         1     34       1   1635.473            80            77  0.522261   
36         1     36       0   4443.018            80            64  0.567061   
38         1     38       1   1725.903           124           110  0.558840   
40         1     40       0   1686.530           128           110  0.657603   
42         1     42       0   2244.929            54            50  0.702233   
44         1     44       1   1361.319           105           110  0.482747   
46         1     46       0   2850.121            96            80  0.698865   
48         1     48       1   1391.672           110           128  0.452632   
50         1     50       1   1582.340           110           119  0.559501   
52         1     52       0   2184.782            58            50  0.581910   
54         1     54       1   2890.742            74            80  0.530453   
56         1     56       1   1901.984            66            80  0.418559   
58         1     58       1   2545.418            94           110  0.576064   
..       ...    ...     ...        ...           ...           ...       ...   
60        33     60       1   4251.426            75            80  0.503628   
62        33     62       0   4601.336            89           110  0.477015   
64        33     64       0   3649.435           110            98  0.416827   
66        33     66       0   3708.628           110            94  0.451137   
68        33     68       1   5089.783            50            54  0.375723   
70        33     70       1   6969.709            50            52  0.424717   
72        33     72       1   1858.269            87           110  0.469035   
74        33     74       1   3910.930            80            96  0.460076   
76        33     76       1   2087.100            50            55  0.532287   
78        33     78       1   4085.180            45            50  0.425232   
80        33     80       1   6549.821            50            53  0.568922   
82        33     82       1   4550.765            80            88  0.449738   
84        33     84       0   5756.064           103           110  0.479955   
86        33     86       0   4035.357            80            72  0.477174   
88        33     88       1   1844.621            80            90  0.497924   
90        33     90       0   3936.449            80            70  0.477459   
92        33     92       0  11964.377            82            80  0.484599   
94        33     94       1   2743.440            96           110  0.529970   
96   

## Hierarchical GLAM estimation

### 1. full GLAM

In [7]:
# Fitting full GLAM
print('Fitting full GLAM individually...')

glam_full = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_PF2019_full'+sufix+'.npy')):
    glam_full.make_model('individual', gamma_bounds=(-1, 1), t0_val=0)
    glam_full.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_full.estimates = np.load(str('results/estimates/glam_PF2019_full'+sufix+'.npy'))   

Fitting full GLAM individually...
Generating single subject models for 32 subjects...


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Fitting 32 model(s) using NUTS...
  Fitting model 1 of 32...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [00:09<00:00, 1210.81draws/s]
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 2 of 32...


...

  Fitting model 31 of 32...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [00:09<00:00, 1223.14draws/s]
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
There were 93 divergences after tuning. Increase `target_accept` or reparameterize.
There were 13 divergences after tuning. Increase `target_accept` or reparameterize.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.4775538612458078, but should be close to 0.8. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


  Fitting model 32 of 32...


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, SNR, gamma, v]
Sampling 4 chains: 100%|██████████| 12000/12000 [00:11<00:00, 1006.88draws/s]
There were 65 divergences after tuning. Increase `target_accept` or reparameterize.
There were 48 divergences after tuning. Increase `target_accept` or reparameterize.
There were 54 divergences after tuning. Increase `target_accept` or reparameterize.
There were 103 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


/!\ Automatically setting parameter precision...


In [8]:
# Save parameter estimates
np.save(str('results/estimates/glam_PF2019_full'+sufix+'.npy'), glam_full.estimates)

In [31]:
pd.DataFrame.from_dict(glam_full.estimates)

0
0   {'b': 1.0, 'p_error': 0.05, 'v': 4.1e-05, 'gam...
1   {'b': 1.0, 'p_error': 0.05, 'v': 1.2e-05, 'gam...
2   {'b': 1.0, 'p_error': 0.05, 'v': 3.1e-05, 'gam...
3   {'b': 1.0, 'p_error': 0.05, 'v': 2.2e-05, 'gam...
4   {'b': 1.0, 'p_error': 0.05, 'v': 3.1e-05, 'gam...
5   {'b': 1.0, 'p_error': 0.05, 'v': 4.3e-05, 'gam...
6   {'b': 1.0, 'p_error': 0.05, 'v': 2.5e-05, 'gam...
7   {'b': 1.0, 'p_error': 0.05, 'v': 5.2e-05, 'gam...
8   {'b': 1.0, 'p_error': 0.05, 'v': 9.5e-05, 'gam...
9   {'b': 1.0, 'p_error': 0.05, 'v': 4e-05, 'gamma...
10  {'b': 1.0, 'p_error': 0.05, 'v': 2e-05, 'gamma...
11  {'b': 1.0, 'p_error': 0.05, 'v': 6.1e-05, 'gam...
12  {'b': 1.0, 'p_error': 0.05, 'v': 1.9e-05, 'gam...
13  {'b': 1.0, 'p_error': 0.05, 'v': 3.8e-05, 'gam...
14  {'b': 1.0, 'p_error': 0.05, 'v': 2.3e-05, 'gam...
15  {'b': 1.0, 'p_error': 0.05, 'v': 4.6e-05, 'gam...
16  {'b': 1.0, 'p_error': 0.05, 'v': 1.7e-05, 'gam...
17  {'b': 1.0, 'p_error': 0.05, 'v': 2.7e-05, 'gam...
18  {'b': 1.0, 'p_error': 0.05, 'v': 3.9e-05, 'gam...
19  {'b': 1.0, 'p_error': 0.05, 'v': 2.7e-05, 'gam...
20  {'b': 1.0, 'p_error': 0.05, 'v': 3.9e-05, 'gam...
21  {'b': 1.0, 'p_error': 0.05, 'v': 4.4e-05, 'gam...
22  {'b': 1.0, 'p_error': 0.05, 'v': 3e-05, 'gamma...
23  {'b': 1.0, 'p_error': 0.05, 'v': 2e-05, 'gamma...
24  {'b': 1.0, 'p_error': 0.05, 'v': 1.7e-05, 'gam...
25  {'b': 1.0, 'p_error': 0.05, 'v': 3.4e-05, 'gam...
26  {'b': 1.0, 'p_error': 0.05, 'v': 3.6e-05, 'gam...
27  {'b': 1.0, 'p_error': 0.05, 'v': 8.4e-05, 'gam...
28  {'b': 1.0, 'p_error': 0.05, 'v': 5.7e-05, 'gam...
29  {'b': 1.0, 'p_error': 0.05, 'v': 4.4e-05, 'gam...
30  {'b': 1.0, 'p_error': 0.05, 'v': 4.3e-05, 'gam...
31  {'b': 1.0, 'p_error': 0.05, 'v': 3.2e-05, 'gam...

In [9]:
# Compute WAICs
print('Computing WAIC scores for full model...')
if not os.path.exists(str('results/waic/glam_PF2019_full'+ sufix +'.npy')):
    # Note: DIC computation does not work for ADVI fitted models
    # But we are using WAIC
    glam_full.compute_waic()
else:
    print('  Found old DIC scores in "results/waic". Skipping WAIC computation...')
    glam_full.waic = np.load(str('results/waic/glam_PF2019_full'+ sufix +'.npy'))

# Compute WAICs
np.save(str('results/waic/glam_PF2019_full'+ sufix +'.npy'), glam_full.waic)

Computing WAIC scores for full model...
  Found old DIC scores in "results/waic". Skipping WAIC computation...


In [22]:
glam_full.waic

array([[1.04604884e+03, 0.00000000e+00, 1.72250244e+00, 1.00000000e+00],
       [1.22067883e+03, 0.00000000e+00, 1.77895232e+00, 1.00000000e+00],
       [1.08131950e+03, 0.00000000e+00, 4.57825376e+00, 1.00000000e+00],
       [1.19129127e+03, 0.00000000e+00, 1.81216772e+00, 1.00000000e+00],
       [1.09360688e+03, 0.00000000e+00, 2.09341500e+00, 1.00000000e+00],
       [1.07141022e+03, 0.00000000e+00, 3.96698860e+00, 1.00000000e+00],
       [1.16532728e+03, 0.00000000e+00, 1.59540281e+00, 1.00000000e+00],
       [1.04997833e+03, 0.00000000e+00, 2.34431071e+00, 1.00000000e+00],
       [9.92213429e+02, 0.00000000e+00, 2.57769666e+00, 1.00000000e+00],
       [1.03406225e+03, 0.00000000e+00, 2.59437231e+00, 1.00000000e+00],
       [1.20205192e+03, 0.00000000e+00, 3.10274965e+00, 1.00000000e+00],
       [1.00093682e+03, 0.00000000e+00, 2.61991370e+00, 1.00000000e+00],
       [1.05702793e+03, 0.00000000e+00, 1.21914822e+01, 1.00000000e+00],
       [1.07529609e+03, 0.00000000e+00, 3.57531961e

In [40]:
# Compute LOO

glam_full.loo = pm.loo(trace=glam_full.trace, model=glam_full.model)
glam_full.loo
np.save(str('results/loo/glam_PF2019_full'+ sufix +'.npy'), glam_full.loo)

AttributeError: 'list' object has no attribute 'nchains'

In [ ]:
glam_full.loo

In [10]:
# Predictions
print('Predicting test set data using full GLAM...')
glam_full.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_PF2019_full'+sufix+'.csv')):
    glam_full.predict(n_repeats=50)
    glam_full.prediction.to_csv(str('results/predictions/glam_PF2019_full'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical full GLAM predictions in "results/predictions". Skipping prediction...')
    glam_full.prediction = pd.read_csv(str('results/predictions/glam_PF2019_full'+sufix+'.csv'))

glam_full.prediction.head()

Predicting test set data using full GLAM...
Replaced attached data (1920 trials) with new data (1920 trials)...


ValueError: Domain error in arguments.

### 1. no-bias GLAM

In [ ]:
# Fitting no-bias GLAM
print('Fitting no-bias GLAM individually...')

glam_nobias = glam.GLAM(train_data)

if not os.path.exists(str('results/estimates/glam_PF2019_nobias'+sufix+'.npy')):
    glam_nobias.make_model('hierarchical', gamma_val=1.0, t0_val=0)
    glam_nobias.fit(method='NUTS', tune=1000)
else:
    print('  Found old parameter estimates in "results/estimates". Skipping estimation...')
    glam_nobias.estimates = np.load(str('results/estimates/glam_PF2019_nobias'+sufix+'.npy'))
 

In [ ]:
   
# Save parameter estimates
np.save(str('results/estimates/glam_PF2019_nobias'+sufix+'.npy'), glam_nobias.estimates)
pd.DataFrame(glam_nobias.estimates)

In [ ]:
# In case it is already fitted
params_part_like = pd.DataFrame.from_dict(glam_nobias.estimates.item(0))
params_part_like

In [ ]:
# Compute LOO

glam_nobias.loo = pm.loo(trace=glam_nobias.trace, model=glam_nobias.model)
glam_nobias.loo

np.save(str('results/loo/glam_PF2019_nobias'+ sufix +'.npy'), glam_nobias.loo
)

In [ ]:
# Predictions
print('Predicting test set data using no-bias GLAM...')
glam_nobias.exchange_data(test_data)

if not os.path.exists(str('results/predictions/glam_PF2019_nobias'+sufix+'.csv')):
    glam_nobias.predict(n_repeats=50)
    glam_nobias.prediction.to_csv(str('results/predictions/glam_PF2019_nobias'+sufix+'.csv'), index=False)
else:
    print('  Found old hierarchical no-bias GLAM predictions in "results/predictions". Skipping prediction...')
    glam_nobias.prediction = pd.read_csv(str('results/predictions/glam_PF2019_nobias'+sufix+'.csv'))

glam_nobias.prediction.head()

## 2. Plot fit

In [11]:
print('Close Figure to continue...')
glam.plot_fit(test_data, [glam_full.prediction]);
#glam.plot_fit(test_data, [glam_full.prediction,glam_nobias.prediction]);

plt.show()

Close Figure to continue...


AttributeError: 'GLAM' object has no attribute 'prediction'

## Parameters for full hierarchical model

In [24]:
params_participant = glam_full.estimates
params_participant

[{'b': 1.0,
  'p_error': 0.05,
  'v': 5.1e-05,
  'gamma': -0.77,
  'SNR': 125.41,
  's': 0.0081,
  'tau': 0.01,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 1.6e-05,
  'gamma': -0.99,
  'SNR': 290.78,
  's': 0.006238,
  'tau': 0.05,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 3.5e-05,
  'gamma': -0.11,
  'SNR': 230.96,
  's': 0.008048,
  'tau': 0.11,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 2.1e-05,
  'gamma': -0.99,
  'SNR': 434.59,
  's': 0.009583,
  'tau': 0.1,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 3.2e-05,
  'gamma': -0.99,
  'SNR': 165.04,
  's': 0.005638,
  'tau': 0.06,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 5.7e-05,
  'gamma': -0.71,
  'SNR': 83.7,
  's': 0.005543,
  'tau': 0.09,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 3e-05,
  'gamma': -0.99,
  'SNR': 232.79,
  's': 0.008216,
  'tau': 0.08,
  't0': array([0.])},
 {'b': 1.0,
  'p_error': 0.05,
  'v': 5.1e-05,
  'gamma

In [21]:
params_participant = pd.DataFrame.from_dict(glam_full.estimates.item(0))

AttributeError: 'list' object has no attribute 'item'

In [ ]:
params_participant

In [ ]:
print ("Mean gamma " +  str(params_participant['gamma'].mean()))

In [ ]:
hist = params_participant[['SNR','gamma','tau','v']].hist(figsize = [20,3] , layout=[1,4],bins = 20)

## [END] 